In [96]:
#!pip install selenium

In [59]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from tqdm import tqdm

In [60]:
url_auctions='https://www.portaldrazeb.cz/drazby/pripravovane'  # page we start from

In [61]:
# initiate the webdriver
driver = webdriver.Chrome('./chromedriver')

# open the link in Chrome
driver.get(url_auctions) 

# make sure page loads
time.sleep(5) 

# we need to click to the page to make the following step to work
driver.find_element_by_css_selector('div[class="el-pagination"]').click() 

# we want to display 100 auctions/page (not the default 20) to decrease scraping time  
el = driver.find_element_by_css_selector('div[class="el-pagination"]').find_element_by_css_selector('i[class="el-select__caret el-input__icon el-icon-arrow-up"]')
time.sleep(5)
el.click()
time.sleep(5)
new_el = el.find_element_by_xpath('//*[ text() = "100/page" ]')
new_el.click()
time.sleep(15)

In [62]:
# get number of pages
last_page = int(soup.find('div',{'class':'el-pagination'}).findAll('li',{'class':'number'})[-1].text) # get number of pages

In [63]:
auctions_pages_soups = []

In [64]:
# getting source codes from all pages and saving as soups
page_number = driver.find_element_by_css_selector('input[type="number"]') # locating element into which we write page number
for page in tqdm(range(1,last_page+1)):
    # get to a page
    page_number.send_keys(Keys.BACK_SPACE)
    page_number.send_keys(Keys.BACK_SPACE)
    page_number.send_keys(str(page))
    page_number.send_keys(Keys.RETURN)
    time.sleep(15)
    
    # save soup object of the page
    html = driver.page_source 
    auctions_pages_soups.append(BeautifulSoup(html, "html"))

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [02:54<00:00, 15.85s/it]


In [65]:
auctions_link_and_category = []

In [66]:
for soup in auctions_pages_soups:
    for i in soup.findAll('article'):
        # extract link
        auction = []
        auction.append(i.find('a')['href'])
        
        # extract categories
        categ = soup.find('article').find('tbody').findAll('tr')[1].find('span').text.lstrip('/').split('/')
        auction.append(categ)
        auctions_link_and_category.append(auction) # save the data

In [70]:
driver.close()

In [67]:
auctions_link_and_category[:5]

[['https://www.portaldrazeb.cz/drazba/054ex633-08-274-dlxd5',
  ['Nemovitosti', 'Pozemek']],
 ['https://www.portaldrazeb.cz/drazba/054ex578-05-302-1ld2g',
  ['Nemovitosti', 'Pozemek']],
 ['https://www.portaldrazeb.cz/drazba/054ex550-04-212-73wpo',
  ['Nemovitosti', 'Pozemek']],
 ['https://www.portaldrazeb.cz/drazba/054ex233-05-313-w8p9z',
  ['Nemovitosti', 'Pozemek']],
 ['https://www.portaldrazeb.cz/drazba/169ex7525-11-164-95llg',
  ['Nemovitosti', 'Pozemek']]]

In [71]:
url_auctioneers = 'https://www.portaldrazeb.cz/drazebnici'

In [77]:
driver = webdriver.Chrome('./chromedriver') 
driver.get(url_auctioneers)  # open the link in Chrome
time.sleep(5)  # make sure page loads
soup = BeautifulSoup(driver.page_source, "html.parser")  # create a soup object from the page source code

In [78]:
auctioneers_soups = []

In [79]:
last_page = int(soup.find('div',{'class':'el-pagination'}).findAll('li',{'class':'number'})[-1].text) # get number of pages
page_number = driver.find_element_by_css_selector('input[type="number"]') # locating the element into which we write page number

In [80]:
for page in tqdm(range(1,last_page+1)):
    # get to a page
    page_number.send_keys(Keys.BACK_SPACE)
    page_number.send_keys(Keys.BACK_SPACE)
    page_number.send_keys(str(page))
    page_number.send_keys(Keys.RETURN)
    
    # make sure page loads
    time.sleep(5)  
    
    # save soups of particular auctioneers
    html = driver.page_source   
    soup = BeautifulSoup(html, "html.parser") # soup of current page  
    for i in soup.findAll('article'):
        auctioneers_soups.append(i) # extract all auctioneers

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:37<00:00,  5.30s/it]


In [81]:
driver.close()

In [96]:
class DataDownloader:
    '''
    This class crawls through dynamic content of https://www.portaldrazeb.cz and collects following things:
            1) soup object for every auctioneer
            2) link to every auction + category of the auction (since the category is not within the auction page itself)
            3) list of all possible categories and regions of auctions (from drop-down menu)
    '''
    def __init__(self):
        self.auctioneers_soups = []
        self.auctions_link_and_category = []
        self.categories = []
        self.regions = []
        print('''Downloader successfully initialized!''')
    def get_soups_of_auctioneers(self,link):
        # initiating a webdriver
        driver = webdriver.Chrome('./chromedriver') 
        
        # opening the link in Chrome
        driver.get(link)  
        
        # making sure page loads
        time.sleep(5) 
        
        # creating a soup object from the page source code
        soup = BeautifulSoup(driver.page_source, "html.parser")  
        
        # getting number of pages
        last_page = int(soup.find('div',{'class':'el-pagination'}).findAll('li',{'class':'number'})[-1].text) 
        
        # locating the element into which we write page number
        page_number = driver.find_element_by_css_selector('input[type="number"]')
        
        # loop through all pages and save the soups
        for page in tqdm(range(1,last_page+1)):
            # get to a page
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(str(page))
            page_number.send_keys(Keys.RETURN)

            # make sure page loads
            time.sleep(5)  

            # save soups of particular auctioneers
            html = driver.page_source   
            soup = BeautifulSoup(html, "html.parser") # soup of current page  
            for i in soup.findAll('article'):
                self.auctioneers_soups.append(i) # extract all auctioneers
                
        # close the window and check soups
        driver.close()
        if len(self.auctioneers_soups)>50:
                print('Soup objects of auctioneers successfully downloaded!')
        

In [97]:
down = DataDownloader()

Downloader successfully initialized!


In [98]:
down.get_soups_of_auctioneers('https://www.portaldrazeb.cz/drazebnici')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:43<00:00,  5.47s/it]

Soup objects of auctioneers successfully downloaded!


In [99]:
len(down.auctioneers_soups)

157